In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://calendar.google.com/calendar/u/0/embed?src=7a04205ae91b85e82ebc74daddbf2933c6b6723b81abb966f0e69c66a996c43b@group.calendar.google.com&ctz=America/Vancouver&pli=1).**

## Imports

In [2]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>
rubric={points:2}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points:2}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
word_list = ["alligator", "giraffe", "candle", "flute"]
glove_wiki_vectors.most_similar(word_list[0])

[('crocodile', 0.7059944868087769),
 ('alligators', 0.6721537709236145),
 ('turtle', 0.6632604598999023),
 ('shark', 0.6062195897102356),
 ('lizard', 0.5981051325798035),
 ('ostrich', 0.5894769430160522),
 ('snake', 0.5842971801757812),
 ('reptile', 0.5779808163642883),
 ('abalone', 0.5678236484527588),
 ('dolphin', 0.5631202459335327)]

In [7]:
glove_wiki_vectors.most_similar(word_list[1])

[('zebra', 0.682654857635498),
 ('hippopotamus', 0.6756220459938049),
 ('giraffes', 0.6284437775611877),
 ('elephant', 0.6181267499923706),
 ('rhinoceros', 0.6120839715003967),
 ('chimp', 0.5907400250434875),
 ('chimpanzee', 0.5900540947914124),
 ('wildebeest', 0.5848949551582336),
 ('parrot', 0.5674313306808472),
 ('cockroach', 0.5665789246559143)]

In [8]:
glove_wiki_vectors.most_similar(word_list[2])

[('candles', 0.791607677936554),
 ('lit', 0.7231093645095825),
 ('flame', 0.6880626678466797),
 ('lamp', 0.6780828237533569),
 ('menorah', 0.6629000902175903),
 ('lighted', 0.6622092723846436),
 ('flickering', 0.6462327837944031),
 ('incense', 0.6436920166015625),
 ('hanukkah', 0.617445170879364),
 ('lamps', 0.5916255116462708)]

In [9]:
glove_wiki_vectors.most_similar(word_list[3])

[('clarinet', 0.8792417049407959),
 ('saxophone', 0.8421716094017029),
 ('cello', 0.8309809565544128),
 ('violin', 0.8177938461303711),
 ('oboe', 0.813706636428833),
 ('trombone', 0.7814450860023499),
 ('trumpet', 0.7771451473236084),
 ('piano', 0.7697390913963318),
 ('percussion', 0.745358943939209),
 ('concerto', 0.7363701462745667)]

In [10]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings
rubric={autograde:2}

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [12]:
coast_shore_similarity = glove_wiki_vectors.similarity(word_pairs[0][0], word_pairs[0][1])
clothes_closet_similarity = glove_wiki_vectors.similarity(word_pairs[1][0], word_pairs[1][1])
old_new_similarity = glove_wiki_vectors.similarity(word_pairs[2][0], word_pairs[2][1])
smart_intelligent_similarity = glove_wiki_vectors.similarity(word_pairs[3][0], word_pairs[3][1])
dog_cat_similarity = glove_wiki_vectors.similarity(word_pairs[4][0], word_pairs[4][1])
tree_lawyer_similarity = glove_wiki_vectors.similarity(word_pairs[5][0], word_pairs[5][1])

...

Ellipsis

In [13]:
grader.check("q1.2")

q1.2 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings
rubric={points:6}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [14]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [15]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [16]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [17]:
glove_wiki_vectors.similarity("black", "rich")

0.5174519

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 6

_Type your answer here, replacing this text._

In [18]:
analogy("woman", "doctor", "man")

woman : doctor :: man : ?


,Analogy word,Score
0,dr.,0.655949
1,brother,0.627400
2,him,0.623644
3,he,0.616981
4,himself,0.607539
5,physician,0.607321
6,father,0.592462
7,master,0.579860
8,friend,0.576395
9,taken,0.573906


In the example, "woman" is to "nurse" as "man" is to "doctor". But, when trying it the other way around, "man" is still similar to "dr." in the same sense that "woman" is similar to "doctor".
The word vectors follow the stereotype that doctors are associated with men and nurses are associated with women. 

In [19]:
print(glove_wiki_vectors.similarity("good", "white"))
print(glove_wiki_vectors.similarity("good", "asian"))
print(glove_wiki_vectors.similarity("good", "black"))
print(glove_wiki_vectors.similarity("good", "hispanic"))

0.486187
0.36140192
0.472803
0.23243824


Certain races are associated with "good"ness more than others. 

In [20]:
print(glove_wiki_vectors.similarity("leadership", "men"))
print(glove_wiki_vectors.similarity("leadership", "women"))

0.39228183
0.41114888


Stereotypically, men are deemed as more suitable to leadership roles, but the similarity between gender and leadership given by the word vectors are the opposite and has close values.

In [21]:
print(glove_wiki_vectors.similarity("intelligence", "democrats"))
print(glove_wiki_vectors.similarity("intelligence", "republicans"))

0.32921758
0.3299697


Stereotypically, democrats are deemed as more intelligent, but the similarity between political sides and intelligence given by word vectors are really close. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion
rubric={points:4}

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

The model probably only de-biased general terms like intelligence and leadership (as seen in the genders vs. leadership and political opinions vs. intelligence examples) , but not specific terms or careers like doctors and computer programmers. Some biases still exist in the pre-trained embeddings, like gender vs. careers and race stereotypes.

The harm of using a biased model in the real world would be reinforcing these stereotypes, causing discrimination and unfair treatment. If companies were to use this embedding model to save time screening applications, ie. in class example, favouring men over women for computer programmer positions, it could dismiss many qualified candidates. 

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 
rubric={autograde:8}

In lecture 18, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [22]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [23]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [24]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [25]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [26]:
#from lecture 17 code31
from sklearn.metrics import classification_report

pipe = make_pipeline(
    CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000)
)
pipe.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = pipe.named_steps["countvectorizer"].transform(X_train)
pipe.fit(X_train, y_train);

bow_train_accuracy = pipe.score(X_train, y_train)
bow_test_accuracy = pipe.score(X_test, y_test)
bow_report = classification_report(y_true=y_test, y_pred=pipe.predict(X_test), digits=2) # use digits=2

print(bow_train_accuracy)
print(bow_test_accuracy)
print(bow_report)
...

0.9562051178314959
0.8173666823973572
                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



Ellipsis

In [27]:
X_train_embeddings = pd.DataFrame([moment.vector for moment in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([moment.vector for moment in nlp.pipe(X_test)])

...

Ellipsis

In [28]:
...

Ellipsis

In [29]:
...

Ellipsis

In [30]:
lgr = LogisticRegression(max_iter=2000)
lgr.fit(X_train_embeddings, y_train)

emb_train_accuracy = lgr.score(X_train_embeddings, y_train)
emb_test_accuracy = lgr.score(X_test_embeddings, y_test)
emb_report = classification_report(y_true=y_test, y_pred=lgr.predict(X_test_embeddings), digits=2) # use digits=2

print(emb_train_accuracy)
print(emb_test_accuracy)
print(emb_report)
...

0.8494993425710529
0.7928268050967437
                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.56      0.51      0.53       469
        exercise       0.66      0.73      0.69        74
         leisure       0.72      0.65      0.68       407
          nature       0.67      0.72      0.69        71

        accuracy                           0.79      4238
       macro avg       0.73      0.73      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



C:\Users\karat\miniconda3\envs\cpsc330\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Ellipsis

In [31]:
grader.check("q1.5")

q1.5 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion
rubric={points:6}

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

_Points:_ 6

1. CountVectorizer converts texts into a matrix where each column is a unique word, focusing on the frequency of word appearances and not word similarities. Average-embedding converts words into vectors that represent the relationships between the words, which helps generalize data better
CountVectorizer performed better than average-embedding in the example, scoring better for precision and f1-score in both macro and weighted averages, and scoring better in weighted average for recall. However, the gap between train and test scores is smaller with average-embedding, so there is less overfitting. 

2. CountVectorizer is more interpretable. The matrix it produces provides the number of occurences for each word, which is straightforward and can be directly interpreted. Average-embedding shows vector representation/floats of word similarities, which requires calculations to interpret.  

3. Yes, we're using transfer learning for the average-embedding approach because it uses pre-trained word embeddings. The benefits of transfer learning is that pre-trained embeddings were already trained with a wide range of dataset, which provides our model with a starting point. Our model can generalize data better especially when the training data we have is small. 


<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [32]:
from sklearn.datasets import fetch_20newsgroups

In [33]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [34]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points:8}

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

_Type your answer here, replacing this text._

In [35]:
# from lecture 17, code60
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["CCONJ", "PUNCT", "DET", "SPACE", "ADV", "PRON", "SYM"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [36]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. 922 sence wonder help u.s.c. ...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq part3 modify 1993/04/04 ----------...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference fed mandate li...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway 4dx-33v try configure need correct...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points:2}

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

The preprocessed texts ignore coordinating conjunctions, punctuations, determiners, spaces, adverbs, pronouns, and symbols. My intention was to keep key terms that would have more impact on the context of the texts (mostly actions/verbs), so words/punctuations that would only impact grammar can be ignored. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points:4}

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [37]:
#from lecture 17, code65
vec = CountVectorizer(stop_words='english')
X_pp = vec.fit_transform(df["text_pp"])

In [38]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 8
lda = LatentDirichletAllocation(
    n_components=n_components, learning_method="batch", max_iter=10, random_state=0
)
document_topics = lda.fit_transform(X_pp)

In [39]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())


#displayed in next question
#import mglearn
#mglearn.tools.print_topics(
#    topics=range(n_components),
#    feature_names=feature_names,
#    sorting=sorting,
#    topics_per_chunk=5,
#    n_words=10,
#)

n_components = 8, because there are 8 target categories in the training dataset according to Exercise 2 introduction ['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points:5}

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [40]:
import mglearn

topics = mglearn.tools.print_topics(
    topics=range(n_components),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)
display(topics)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
file          game          armenian      god           window        
image         team          turkish       jesus         widget        
edu           play          jews          believe       55            
program       season        armenians     think         use           
available     hockey        israel        know          application   
use           player        people        people        display       
include       year          turkey        say           problem       
version       win           israeli       question      work          
output        period        war           bible         set           
ftp           goal          turks         thing         server        


topic 5       topic 6       topic 7       
--------      --------      --------      
year          people        file          
good          say

None

topic 0: computer programs, topic 1: hockey , topic 2: people/politics, topic 3: christianity, topic 4: windows OS, topic 5: sports, topic 6: beliefs, topic 7: law

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points:5}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [41]:
topic_titles = ["computer programs", "hockey", "people/politics", "christianity", "windows OS", "sports", "beliefs", "law"]

for i in range(5):
    print("Document", i, ": ")
    display(df.iloc[i,0])
    # document_topics[i] is topic distribution
    cat_number = np.argmax(document_topics[i])
    print("Topic assignment for document: ", cat_number, " ", topic_titles[cat_number])  
    print(" ")

Document 0 : 


'You know, I was reading 18 U.S.C. 922 and something just did not make \nsence and I was wondering if someone could help me out.\n\nSay U.S.C. 922 :\n\n(1) Except as provided in paragraph (2), it shall be unlawful for\nany person to transfer or possess a machinegun.\n\n Well I got to looking in my law dictionary and I found that a "person" \nmight also be an artificial entity that is created by government \nand has no rights under the federal constitution. So, what I \ndon\'t understand is how a statute like 922 can be enforced on \nan individual. So someone tell me how my government can tell\nme what I can or cannot possess. Just passing a law \ndoes not make it LAW. Everyone knows that laws are constitional\nuntil it goes to court. So, has it ever gone to court, not\njust your run of the mill "Ok I had it I am guilty, put me in jail"\n\nHas anyone ever claimed that they had a right to possess and was told\nby the Supreme Court that they didn\'t have that right?\n\n\n'

Topic assignment for document:  6   beliefs
 
Document 1 : 


"\n\n\nIt's not a bad question: I don't have any refs that list this algorithm\neither. But thinking about it a bit, it shouldn't be too hard.\n\n1) Take three of the points and find the plane they define as well as\nthe circle that they lie on (you say you have this algorithm already)\n\n2) Find the center  of this circle. The line passing through this center\nperpendicular to the plane of the three points passes through the center of\nthe sphere.\n\n3) Repeat with the unused point and two of the original points. This\ngives you two different lines that both pass through the sphere's\norigin. Their interection is the center of the sphere.\n\n4) the radius is easy to compute, it's just the distance from the center to\nany of the original points.\n\nI'll leave the math to you, but this is a workable algorithm. :-)\n\n\nAn alternate method would be to take pairs of points: the plane formed\nby the perpendicular bisector of each line segment pair also contains the\ncenter of the sphere. T

Topic assignment for document:  0   computer programs
 
Document 2 : 


...

Topic assignment for document:  3   christianity
 
Document 3 : 


"The following problem is really bugging me,\nand I would appreciate any help.\n\nI create two windows:\n\nw1 (child to root) with event_mask = ButtonPressMask|KeyPressMask;\nw2 (child to w1) with do_not_propagate_mask = ButtonPressMask|KeyPressMask;\n\n\nKeypress events in w2 are discarded, but ButtonPress events fall through\nto w1, with subwindow set to w2.\n\nFYI, I'm using xnews/olvwm.\n\nAm I doing something fundamentally wrong here?"

Topic assignment for document:  4   windows OS
 
Document 4 : 


"\n\n  This is the latest from UPI \n\n     Foreign Ministry spokesman Ferhat Ataman told journalists Turkey was\n     closing its air space to all flights to and from Armenia and would\n     prevent humanitarian aid from reaching the republic overland across\n     Turkish territory.\n\n  \n   Historically even the most uncivilized of peoples have exhibited \n   signs of compassion by allowing humanitarian aid to reach civilian\n   populations. Even the Nazis did this much.\n\n   It seems as though from now on Turkey will publicly pronounce \n   themselves 'hypocrites' should they choose to continue their\n   condemnation of the Serbians.\n\n\n\n--"

Topic assignment for document:  2   people/politics
 


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points:6}

1. In lecture 19, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 19 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. 

Decision trees: automatically handles multi-class classification. It might implicitly using One vs. Rest approach, splitting data into one of the two classes (class 1 vs. not 1, class 2 vs. not 2) at each node judging by some feature/threshold, then do a majority vote at the end to select the most appropriate class. 

KNN: automatically handles multi-class classification. KNN splits data using a decision boundary, might implicitly One vs. One approach, then do a majority vote. Since KNN does not perform well with large subsets (more irrelevant data), focusing on just the subsets (class 1 vs. class 2, class 2 vs. class 3) instead of using all the dataset like One vs. Rest is better.

Random forest: automatically handles multi-class classification. It splits data by considering a randomized set of features and might handle multiclass classification implicitly using One vs. Rest approach like with decision trees, since random forests perform well generally and One vs. Rest is faster. 

Logistic regression: splits data into groups using a decision boundary. It uses One vs. Rest or multinomial approach according to the given multi_class parameter (auto, ovr, multinomial)

SVM RBF: splits data into two groups with a decision boundary. It could use One vs. One or One vs. Rest (by specifying the parameter decision_function_shape), depending on if we want accuracy or speed. 

2.

Transfer learning is using previous learned knowledge on a task. In natural language processing, it's using pre-trained models (eg. synonyms, contexts) that have been trained on previous datasets on new dataset. It learns the similarities and connotations of words. The advantage is saving time training data, where we only need to fine-tune our model, and helps with increasing generality especially if we have a small training data. The disadvantage is we have less control over the model and the underlying biases, and it's harder to figure out the biases and de-bias the model. 

3. 

We have more flexibility and can customize/fine-tune parameters, so we don't need to use the pre-trained model out of the box. We can choose to use it as feature transformers and train our own models.


<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)